In [8]:
import sympy as sp
from sympy import Matrix

# How to calculate the Jordan normal form of a matrix

Consider the following matrix

In [9]:
A = Matrix([
    [5,1,-2,4],
    [0,5,2,2],
    [0,0,5,3],
    [0,0,0,4]
])
A

Matrix([
[5, 1, -2, 4],
[0, 5,  2, 2],
[0, 0,  5, 3],
[0, 0,  0, 4]])

To find the Jordan canonical form and the change of basis matrix, first, we need to calculate the spectrume of the matrix

In [10]:
A.eigenvals()

{4: 1, 5: 3}

Then for each of these eigenvalues we need to find the generalized eigenspaces. For the case where $ \lambda=4 $, since the multiplicity of the eigenvalue is one, the generalized eigenspace is the same as the eigenspace spanned by the classic eigenvectors. This is the same as of $ ker(A-\lambda I) $.

In [12]:
I = sp.eye(4)
(A-4*I).nullspace()[0]

Matrix([
[-14],
[  4],
[ -3],
[  1]])

Now we consider the more interesting case $ \lambda = 5 $. In this case the multiplicity is 3. First, we compute the eigenvectors corresponding to this eigenvalue. The eigenvector is
$$ v = (1,0,0,0)^T $$

In [20]:
I = sp.eye(4)
vees = (A-5*I).nullspace()
vees[0]

Matrix([
[1],
[0],
[0],
[0]])

There is only one of them (we were expecting to see three!). So the matrix is defficient and the geometric multiplicity of $ \lambda $ (that is 1, which corresponds to one eigenvector found above) does not match with the algebraic multiplicity (that is 3, which is the algebraic multiplicity of the root).

Here, we present two ways to perform the calculation:

### First method

To find the generalized eigenvectors, we need to find the Jordan chain for this eigenvector. The first generalized eigenvector $ v' $ can be found by solving

$$ (A - \lambda I)v' = v. $$

In [22]:
v = vees[0]
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
sol = sp.linsolve(((A-5*I), v), x1, x2, x3, x4)
print(sol)

{(x1, 1, 0, 0)}


There is no restriction on $ x_1 $. We set it to be zero. So we now have $ v' $ as below
$$ v' = (0,1,0,0)^T $$

In [42]:
v_p = Matrix([0,1,0,0])

Furthermore, to calculate the second generalized eigenvector, we need to solve
$$  (A-\lambda I)v'' = v'.  $$

In [43]:
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
sol = sp.linsolve(((A-5*I), v_p), x1, x2, x3, x4)
print(sol)

{(x1, 1, 1/2, 0)}


Again, there is no restriction on $ x_1 $ and we choos it to be $1$. So we have calculated
$$ v'' = (0,1,1/2,0) $$

Now we can construct the change of basis matrix $P$ that can transform that matrix $A$ to its Jordan block form. The columns of this matrix is the generalized eigenvectors

In [52]:
P = Matrix([
    [-14, 1, 0, 0],
    [4, 0, 1, 1],
    [-3, 0, 0, 1/2],
    [1, 0, 0, 0]
])

So we can verify all our calculation above by

In [54]:
(P.inv() * A * P).applyfunc(lambda x: round(x, 2))

Matrix([
[4.0,   0,   0,   0],
[0.0, 5.0, 1.0, 0.0],
[0.0,   0, 5.0, 1.0],
[  0,   0,   0, 5.0]])

### Second method

In the second method we first find the highest rank generalized eigenvector and from that we find the lower rank ones. I.e. we want to find $w$ such that
$$ (A-\lambda I)^3w = 0, \qquad \text{and} \qquad (A-\lambda I)^2w \neq 0. $$

Why power 3? Because that is the first power that the matrix $ (A-\lambda I)^3 $ has rank $1= n - \mu$, where $\mu$ is the algebraic multiplicity of the eigenvalue (so $n=4$, and $\mu=3$).

In [57]:
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
sol = sp.linsolve(((A-5*I)**3, v*0), x1, x2, x3, x4)
print(sol)

{(x1, x2, x3, 0)}


In [58]:
sol = sp.linsolve(((A-5*I)**2, v*0), x1, x2, x3, x4)
print(sol)

{(x1, x2, 0, 0)}


So we need to have $x_3 \neq 0$ and $x_4=0$. There are no further restrictions on $x_2$, and $x_1$. One possible chice is

$$ w = (0,0,1,0). $$

In [80]:
w = Matrix([0,0,1,0])
# w = Matrix([1,1,1/2,0])

So the second generalized eigenvector will be
$$ w_p = (-2,2,0,0) $$

In [81]:
w_p = (A - 5*I) * w
w_p

Matrix([
[-2],
[ 2],
[ 0],
[ 0]])

and lastly, the last generalized eigenvector will be
$$ w_p = (2,0,0,0) $$

In [82]:
w_pp = (A - 5*I) * w_p
w_pp

Matrix([
[2],
[0],
[0],
[0]])

And to double check the calculations we can calculate the change of basis matrix

In [83]:
P = Matrix([
    [-14, 0, -2, 2],
    [4, 0, 2, 0],
    [-3, 1, 0, 0],
    [1, 0, 0, 0]
])

In [84]:
P.inv() * A * P

Matrix([
[4, 0, 0, 0],
[0, 5, 0, 0],
[0, 1, 5, 0],
[0, 0, 1, 5]])

# Another Example

`Important Note`: In the following, something strange happened. As usuall, I first calculated the eigenvalues and then eigenvectors and using each eigenvector I started to find the Jordan chain, hence the generalized eigenvectors, hence the generalized eigenspaces. Bue with the choice of $v_3$ as below, we can not find all three generalized eigenvectors (one of them is the classic eigenvalue). So then I used another eigenvector that I found by inspecting the Jordan form of this matrix using `A.jordan_form()`

In [191]:
A = Matrix([[1,0,0,0,0,0],[0,0,0,0,-1,1],[-1,-1,1,1,-1,1],[0,0,0,1,0,0],[0,1,0,0,2,0],[0,0,0,0,0,1]])
I = sp.eye(6)
A

Matrix([
[ 1,  0, 0, 0,  0, 0],
[ 0,  0, 0, 0, -1, 1],
[-1, -1, 1, 1, -1, 1],
[ 0,  0, 0, 1,  0, 0],
[ 0,  1, 0, 0,  2, 0],
[ 0,  0, 0, 0,  0, 1]])

First, let's calculate the eigenvalues. We have one eigenvalue $\lambda = 1$ with multiplicity 6.

In [273]:
A.eigenvals()


{1: 6}

Now we calculate the eigenvectors corresponding to this eigenvalue

In [95]:
vees = (A-I).nullspace()
vees 

[Matrix([
 [0],
 [0],
 [1],
 [0],
 [0],
 [0]]),
 Matrix([
 [1],
 [0],
 [0],
 [1],
 [0],
 [0]]),
 Matrix([
 [ 0],
 [-1],
 [ 0],
 [ 0],
 [ 1],
 [ 0]])]

So we have three eigenvalues
$$ v_1 = (0,0,1,0,0,0)^T, \quad v_2=(1,0,0,1,0,0)^T, \quad v_3=(0,-1,0,0,1,0)^T $$

In [275]:
v1 = vees[0]
v2 = vees[1]
v3 = vees[2]  ## This eigen value is not working properly and needs to be replaced with another one later.

Now we aim to calculate the generalized eigenvectors by computing the Jordan chain for each of vectors above. We start with $v_1$.

We need to find $v_1'$ that satisfies
$$ (A-I)v_1' = v_1. $$

In [109]:
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6')
sol = sp.linsolve(((A-I), v1), x1, x2, x3, x4, x5, x6)
print(sol)

{(x4 - 1, -x5, x3, x4, x5, 0)}


By choosing $x_3 = x_4 = x_5 = 0 $ we will get

$$ v_1' = (-1,0,0,0,0,0)^T. $$

In [124]:
vp_1 = Matrix([-1,0,0,0,0,0])

We continue bulding the Jordan chain. So we aim at calculating $v''_1$ such that
$$ (A-I)v''_1 = v'_1. $$

In [125]:
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6')
sol = sp.linsolve(((A-I), vp_1), x1, x2, x3, x4, x5, x6)
print(sol)

EmptySet


So there is no further solution and the jordan chain terminates for this eigenvector. 

We then start with the second eigenvector
$$  v_2 = (1,0,0,1,0,0)^T $$

Similar to above, we solve the equation
$$ (A-I)v'_2 = v_2. $$

In [240]:
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6')
sol = sp.linsolve(((A-I), v2), x1, x2, x3, x4, x5, x6)
print(sol)

EmptySet


There is no further generalized eigenvectors.

Finally, we calculate the generalized eigenvectors for $v_3$. Similar to above we find $v'_3$ by solving
$$ (A-I)v'_3 = v_3. $$

In [267]:
## I don't know why, the eigenvector detected above (v3), is not working well and you can not find all the generalized eigenvectors. 
## Through trial and error I found the following eigenvector that seems to be working well to generate all eigenspaces
v3 = Matrix([0,-1,-1,0,1,0])
v3

Matrix([
[ 0],
[-1],
[-1],
[ 0],
[ 1],
[ 0]])

In [264]:
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6')
sol = sp.linsolve(((A-I), v3), x1, x2, x3, x4, x5, x6)
print(sol)

{(x4, 1 - x5, x3, x4, x5, 0)}


by choosing $x_4 = x_5 = x_3 = 0$ we will get
$$ v'_3 = (0,1,0,0,0,0). $$

In [265]:
# vp_3 = Matrix([0,0,1,1,1,0])
vp_3 = Matrix([0,1,0,0,0,0])

In search for more generalized eigenvectors, we solve

In [266]:
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6')
sol = sp.linsolve(((A-I), vp_3), x1, x2, x3, x4, x5, x6)
print(sol)

{(x4 + 1, -x5, x3, x4, x5, 1)}


So by letting $ x_4 = x_3 = x_5 = 0 $, we will get
$$ v_3'' = (1,0,0,0,0,1) $$

In summary the generalized eigenvectors are
$$ v_1 = (0,0,1,0,0,0),\quad  v'_1=(-1,0,0,0,0,0) $$
$$ v_2 = (1,0,0,1,0,0)$$
$$ v_3 = (0,-1,-1,0,1,0), \qquad v'_3 = (0,1,0,0,0,0), \qquad v''_3 = (1,0,0,0,0,1).$$

And we can verify there is no more generalized eigenvectors

In [269]:
vpp_3 = Matrix([1,0,0,0,0,1])
x1, x2, x3, x4, x5, x6 = sp.symbols('x1 x2 x3 x4 x5 x6')
sol = sp.linsolve(((A-I), vpp_3), x1, x2, x3, x4, x5, x6)
print(sol)

EmptySet


We can validate our computation by constructing the change of basis matrix

In [271]:
P = Matrix([
    [0,-1,1,0,0,1],
    [0,0,0,-1,1,0],
    [1,0,0,-1,0,0],
    [0,0,1,0,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,1]
])
P

Matrix([
[0, -1, 1,  0, 0, 1],
[0,  0, 0, -1, 1, 0],
[1,  0, 0, -1, 0, 0],
[0,  0, 1,  0, 0, 0],
[0,  0, 0,  1, 0, 0],
[0,  0, 0,  0, 0, 1]])

In [272]:
P.inv() * A * P

Matrix([
[1, 1, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0],
[0, 0, 0, 1, 1, 0],
[0, 0, 0, 0, 1, 1],
[0, 0, 0, 0, 0, 1]])

# Yet Another Example

In [322]:
A = Matrix([
    [0,1,-1,0],
    [0,0,0,-1],
    [1,0,0,1],
    [0,1,0,0]
])
I = sp.eye(4)

In [323]:
A.eigenvals()

{-I: 2, I: 2}

In [324]:
lam = 1j
vees = (A-I*lam).nullspace()
vees

[Matrix([
 [1.0*I],
 [    0],
 [    1],
 [    0]])]

In [325]:
v = vees[0]
v

Matrix([
[1.0*I],
[    0],
[    1],
[    0]])

In [326]:
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
sol = sp.linsolve(((A-I*lam), v), x1, x2, x3, x4)
print(sol)

{(1.0*I*x3, 1.0*I, x3, 1.0)}


So
$$ v_1^1 = (i,0,1,0), \qquad v_1^2 = (0,i,0,1) $$

and for the second eigenvector

In [327]:
lam = -1j
vees = (A-I*lam).nullspace()
vees

[Matrix([
 [-1.0*I],
 [     0],
 [     1],
 [     0]])]

In [328]:
v = vees[0]
v

Matrix([
[-1.0*I],
[     0],
[     1],
[     0]])

In [329]:
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')
sol = sp.linsolve(((A-I*lam), v), x1, x2, x3, x4)
print(sol)

{(-1.0*I*x3, -1.0*I, x3, 1.0)}


So
$$ v_2^1 = (-i,0,1,0), \qquad v_2^2 = (0,-i,0,1) $$

Then the change of basis matrix will be


In [330]:
P = Matrix([
    [1j,0,-1j,0],
    [0,1j,0,-1j],
    [1,0,1,0],
    [0,1,0,1]
])

In [331]:
P.inv() * A * P

Matrix([
[1.0*I,   1.0,      0,      0],
[    0, 1.0*I,      0,      0],
[    0,     0, -1.0*I,    1.0],
[    0,     0,      0, -1.0*I]])